In [1]:
from urllib import request
from nltk.tokenize import sent_tokenize
from arrow import now

roots = [13897, 59297, 72782, ]

URL = 'https://www.gutenberg.org/cache/epub/{}/pg{}.txt'.format(roots[0], roots[0])

time_start = now()
lines = [line.decode('utf-8') for line in request.urlopen(url=URL)]
sentences = sent_tokenize(text=' '.join(lines).replace('\n', ' ').replace('\r', ' '), language='english')
print('done in {}; we have {} lines and {} sentences'.format(now() - time_start, len(lines), len(sentences)))

done in 0:00:00.867891; we have 7262 lines and 4551 sentences


We have the text from our URL rendered as sentences. 

In [2]:
import pandas as pd
from arrow import now
from transformers import pipeline

MODEL = 'bhadresh-savani/distilbert-base-uncased-emotion'

# this takes about a minute per thousand sentences
time_start = now()
pipe = pipeline(task='sentiment-analysis', model=MODEL)
result_df = pd.DataFrame(pipe(sentences)).groupby(by='label').sum().reset_index()

print('done in {}'.format(now() - time_start))

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

done in 0:03:03.307983


Now we have our aggregate sentiment score. Let's visualize it as a radar plot.

In [3]:
from plotly.express import line_polar
line_polar(data_frame=result_df, r='score', theta='label', line_close=True, title=lines[10])